In [1]:
import sys
path = ".."
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
import numpy as np
from data_retrieval import lipade_groundtruth
from clustering import getPredictionFromThreshold
import evaluators

corpus = "lipade_groundtruth"
type = "image"
method = "sift_absolute"

distancePath = "../representation/" + type + "/results/distance/" + corpus + "/" + method + ".npy"
distance = np.load(distancePath)

In [3]:
_,_,y = lipade_groundtruth.getDataset()

In [ ]:
thresholds = np.linspace(0, 1, 1000) # Can be adapted for some methodes (i.e. SIFT : (0.95, 1, 1000))

precisions, recalls, f1s = evaluators.p_r_f1_byThresholds(thresholds, distance, y)

AP, bestThresholdIndex = evaluators.pr_curve(precisions, recalls, f1s, other=("Fei mAP", evaluators.fei_mAP(y, distance)))

precisions_per_class, recalls_per_class = evaluators.p_r_class_byThresholds(thresholds, distance, y)

print(evaluators.goncalves_mAP(precisions_per_class, recalls_per_class))

In [ ]:
bestPredictedY = getPredictionFromThreshold(thresholds[bestThresholdIndex], distance)
np.save("clusters/" + corpus + "/" + method + ".npy", bestPredictedY)